In [ ]:
import os
import numpy as np
from PIL import Image
import matplotlib.pyplot as plt
import matplotlib.patches as patches

# Function to convert mask to a single bounding box
def mask_to_bbox(mask):
    # Find non-zero elements (tumor regions)
    rows = np.any(mask, axis=1)
    cols = np.any(mask, axis=0)

    if np.any(rows) and np.any(cols):  # Check if there's any non-zero area in the mask
        y_min, y_max = np.where(rows)[0][[0, -1]]
        x_min, x_max = np.where(cols)[0][[0, -1]]
        return [x_min, y_min, x_max, y_max]
    else:
        return None  # Return None if the mask is empty

# Function to save bounding box annotations
def save_annotations(image_path, bbox, annotation_dir, class_id):
    if bbox is None:  # Skip if no bounding box is detected
        return

    image = Image.open(image_path)
    w, h = image.size

    # Calculate normalized center, width, and height of the bounding box
    x_center = (bbox[0] + bbox[2]) / 2.0 / w
    y_center = (bbox[1] + bbox[3]) / 2.0 / h
    width = (bbox[2] - bbox[0]) / w
    height = (bbox[3] - bbox[1]) / h

    annotation = f"{class_id} {x_center:.6f} {y_center:.6f} {width:.6f} {height:.6f}"

    # Create corresponding subdirectory in annotation_dir
    rel_dir = os.path.relpath(os.path.dirname(image_path), root_dir)
    annotation_subdir = os.path.join(annotation_dir, rel_dir)
    os.makedirs(annotation_subdir, exist_ok=True)

    image_name = os.path.splitext(os.path.basename(image_path))[0]
    annotation_path = os.path.join(annotation_subdir, image_name + ".txt")

    with open(annotation_path, 'w') as f:
        f.write(annotation)

# Function to visualize annotations with class names
def visualize_annotations(image_path, bbox, class_name):
    image = Image.open(image_path)
    fig, ax = plt.subplots(1, figsize=(12, 8))
    ax.imshow(image)

    if bbox is not None:
        rect = patches.Rectangle((bbox[0], bbox[1]), bbox[2] - bbox[0], bbox[3] - bbox[1],
                                 linewidth=2, edgecolor='r', facecolor='none')
        ax.add_patch(rect)
        ax.text(bbox[0], bbox[1] - 10, class_name, color='red', fontsize=12, backgroundcolor='white')

    plt.axis('off')
    plt.show()

# Function to process the dataset and create annotations
def process_dataset(root_dir, annotation_dir):
    class_names = {
        "0": "No Tumor",
        "1": "Glioma",
        "2": "Meningioma",
        "3": "Pituitary"
    }

    for cls in os.listdir(os.path.join(root_dir, 'image')):  # Iterate over each class directory
        cls_img_dir = os.path.join(root_dir, 'image', cls)
        cls_mask_dir = os.path.join(root_dir, 'mask', cls)

        cls_images = sorted([img for img in os.listdir(cls_img_dir) if os.path.isfile(os.path.join(cls_img_dir, img))])

        class_id = int(cls)  # Convert folder name (0, 1, 2, 3) to class_id
        class_name = class_names[cls]  # Get the class name

        # Ensure images and masks are aligned
        for img in cls_images:
            base_name = img.split('.')[0]
            mask_name = f"{base_name}_m.jpg"
            img_path = os.path.join(cls_img_dir, img)
            mask_path = os.path.join(cls_mask_dir, mask_name)

            if os.path.exists(mask_path):
                mask = Image.open(mask_path).convert("L")  # Load mask as grayscale
                mask = np.array(mask)  # Convert mask to numpy array

                # Detect the single bounding box
                bbox = mask_to_bbox(mask)

                # Save bounding box annotations
                save_annotations(img_path, bbox, annotation_dir, class_id)

                # Visualize annotations
                visualize_annotations(img_path, bbox, class_name)
            else:
                print(f"No corresponding mask found for image: {img_path}")

# Define paths
root_dir = '/user/HS402/rk01219/Downloads/Segmentation/archive/Brain Tumor Segmentation Dataset'
annotation_dir = '/user/HS402/rk01219/Downloads/annotations'  # Directory to save annotations

# Process the dataset to create annotations
process_dataset(root_dir, annotation_dir)


In [ ]:
import os
from collections import defaultdict

def load_image_and_annotations(image_dir, annotation_dir):
    """
    Load images and their corresponding annotations, and count the number of images per tumor type.

    :param image_dir: Directory where the images are saved.
    :param annotation_dir: Directory where the annotation files are saved.
    :return: Dictionary with the number of images and matching annotations per tumor type.
    """
    tumor_image_counts = defaultdict(int)
    tumor_annotation_counts = defaultdict(int)
    class_names = {
        "0": "No Tumor",
        "1": "Glioma",
        "2": "Meningioma",
        "3": "Pituitary"
    }

    # Iterate over all subdirectories in the image directory
    for cls in os.listdir(image_dir):
        cls_img_dir = os.path.join(image_dir, cls)
        cls_annotation_dir = os.path.join(annotation_dir, 'image', cls)  # Adjusted to match the new structure

        if not os.path.isdir(cls_img_dir):
            continue

        # Count images in the image directory
        images = [img for img in os.listdir(cls_img_dir) if img.endswith(('.png', '.jpg'))]
        tumor_image_counts[class_names[cls]] += len(images)

        # Count annotations in the corresponding annotation directory
        if os.path.exists(cls_annotation_dir):
            annotations = [ann for ann in os.listdir(cls_annotation_dir) if ann.endswith('.txt')]
            tumor_annotation_counts[class_names[cls]] += len(annotations)

    return tumor_image_counts, tumor_annotation_counts

def print_tumor_image_and_annotation_counts(tumor_image_counts, tumor_annotation_counts):
    """
    Print the number of images and matching annotations for each tumor type.

    :param tumor_image_counts: Dictionary with the number of images per tumor type.
    :param tumor_annotation_counts: Dictionary with the number of annotations per tumor type.
    """
    print("Number of images and annotations per tumor type:")
    for tumor_type in tumor_image_counts.keys():
        image_count = tumor_image_counts[tumor_type]
        annotation_count = tumor_annotation_counts.get(tumor_type, 0)
        print(f"{tumor_type}: Images = {image_count}, Annotations = {annotation_count}")

# Define the image and annotation directory paths
image_dir = '/user/HS402/rk01219/Downloads/Segmentation/archive/Brain Tumor Segmentation Dataset/image'
annotation_dir = '/user/HS402/rk01219/Downloads/annotations'  # Directory containing the annotations

# Load the images and annotations and count the images and annotations per tumor type
tumor_image_counts, tumor_annotation_counts = load_image_and_annotations(image_dir, annotation_dir)

# Print the results
print_tumor_image_and_annotation_counts(tumor_image_counts, tumor_annotation_counts)
